In [ ]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from backtesting import Backtest
import os
import warnings
warnings.filterwarnings('ignore')

from strategies.strat_daily_mr import StratDailyMR

print("Imports successful")

c:\Users\romai\OneDrive\Documents\day trading\mon_bot_propre\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\romai\OneDrive\Documents\day trading\mon_bot_propre\.venv\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

✅ All imports successful!


# IWM Trading Strategy Analysis
Daily Mean Reversion Strategy using RSI(2) and SMA(200) trend filter

In [ ]:
csv_file = os.path.join("data", "IWM-D1-2010-2025.csv")

try:
    df_iwm = pd.read_csv(csv_file, skiprows=[0,1])
    df_iwm.columns = ['Time', 'Close', 'High', 'Low', 'Open', 'Volume']
    df_iwm['Time'] = pd.to_datetime(df_iwm['Time'])
    df_iwm.set_index('Time', inplace=True)
    df_iwm = df_iwm[['Open', 'High', 'Low', 'Close', 'Volume']]
    df_iwm = df_iwm.apply(pd.to_numeric, errors='coerce')
    df_iwm = df_iwm[~df_iwm.index.duplicated(keep='first')]
    df_iwm.dropna(inplace=True)
    
    print(f"Data loaded: {len(df_iwm)} candles from {df_iwm.index[0]} to {df_iwm.index[-1]}")
    print(f"Shape: {df_iwm.shape}")
    df_iwm.tail(3)
except FileNotFoundError:
    print("Error: File not found")

✅ IWM Data loaded: 3986 candles from 2010-01-04 00:00:00 to 2025-11-05 00:00:00
Data shape: (3986, 5)


In [ ]:
bt_iwm = Backtest(df_iwm, StratDailyMR, cash=10000, commission=.001)
stats_iwm = bt_iwm.run()

print("\n" + "="*60)
print("BASELINE STRATEGY RESULTS")
print("="*60)
print(stats_iwm)
print("="*60)


📈 STRATEGY 1: DAILY MEAN REVERSION (IWM)
Start                     2010-01-04 00:00:00
End                       2025-11-05 00:00:00
Duration                   5784 days 00:00:00
Exposure Time [%]                     13.3718
Equity Final [$]                  10009.21678
Equity Peak [$]                   10023.14006
Commissions [$]                      28.84496
Return [%]                            0.09217
Buy & Hold Return [%]               324.24426
Return (Ann.) [%]                     0.00582
Volatility (Ann.) [%]                 0.10682
CAGR [%]                              0.00401
Sharpe Ratio                          0.05453
Sortino Ratio                         0.07423
Calmar Ratio                          0.01714
Alpha [%]                            -0.37933
Beta                                  0.00145
Max. Drawdown [%]                    -0.33977
Avg. Drawdown [%]                     -0.0521
Max. Drawdown Duration     1518 days 00:00:00
Avg. Drawdown Duration      270 days 0

## Enhanced Strategy with Multiple Filters
Testing volume, volatility, and day-of-week filters

In [ ]:
from backtesting import Strategy

class StratDailyMR_Enhanced(Strategy):
    
    trend_sma_period = 200
    rsi_period = 2
    rsi_oversold_level = 10
    exit_sma_period = 5
    fixed_size = 1.0
    
    volume_ma_period = 20
    use_volume_filter = True
    volume_multiplier = 1.0
    
    use_volatility_filter = False
    atr_period = 14
    atr_threshold = 0.5
    
    use_day_filter = False
    avoid_monday = False
    avoid_friday = False
    
    def init(self):
        from pandas_ta import sma, rsi, atr
        
        close = pd.Series(self.data.Close, index=self.data.index)
        high = pd.Series(self.data.High, index=self.data.index)
        low = pd.Series(self.data.Low, index=self.data.index)
        volume = pd.Series(self.data.Volume, index=self.data.index)
        
        self.sma_trend = self.I(lambda x: sma(pd.Series(x), length=self.trend_sma_period).values, self.data.Close)
        self.rsi = self.I(lambda x: rsi(pd.Series(x), length=self.rsi_period).values, self.data.Close)
        self.sma_exit = self.I(lambda x: sma(pd.Series(x), length=self.exit_sma_period).values, self.data.Close)
        
        if self.use_volume_filter:
            self.volume_ma = self.I(lambda x: sma(pd.Series(x), length=self.volume_ma_period).values, self.data.Volume)
        
        if self.use_volatility_filter:
            self.atr = self.I(lambda h, l, c: atr(pd.Series(h), pd.Series(l), pd.Series(c), length=self.atr_period).values, 
                            self.data.High, self.data.Low, self.data.Close)

    def next(self):
        
        if len(self.rsi) < max(self.rsi_period, self.trend_sma_period, self.exit_sma_period):
            return

        current_close = self.data.Close[-1]
        current_rsi = self.rsi[-1]
        current_trend_sma = self.sma_trend[-1]
        current_exit_sma = self.sma_exit[-1]
        
        if self.position:
            if current_close > current_exit_sma:
                self.position.close()
                return
        
        if not self.position:
            is_uptrend = current_close > current_trend_sma
            is_oversold = current_rsi < self.rsi_oversold_level
            
            volume_ok = True
            if self.use_volume_filter and len(self.volume_ma) >= self.volume_ma_period:
                volume_ok = self.data.Volume[-1] > (self.volume_ma[-1] * self.volume_multiplier)
            
            volatility_ok = True
            if self.use_volatility_filter and len(self.atr) >= self.atr_period:
                volatility_ok = self.atr[-1] > self.atr_threshold
            
            day_ok = True
            if self.use_day_filter:
                current_day = self.data.index[-1].dayofweek
                if self.avoid_monday and current_day == 0:
                    day_ok = False
                if self.avoid_friday and current_day == 4:
                    day_ok = False
            
            if is_uptrend and is_oversold and volume_ok and volatility_ok and day_ok:
                self.buy(size=self.fixed_size)

print("Enhanced strategy created")

✅ Enhanced strategy created!


In [ ]:
bt_enhanced = Backtest(df_iwm, StratDailyMR_Enhanced, cash=10000, commission=.001)

print("Testing base strategy...")
stats_enhanced_base = bt_enhanced.run()

print("\nTesting with volume filter...")
stats_enhanced_vol = bt_enhanced.run(use_volume_filter=True, volume_multiplier=1.0)

print("\nTesting with higher volume threshold...")
stats_enhanced_vol_high = bt_enhanced.run(use_volume_filter=True, volume_multiplier=1.5)

print("\n" + "="*70)
print("FILTER IMPACT COMPARISON")
print("="*70)

comparison_filters = pd.DataFrame({
    'Original': stats_iwm,
    'Enhanced Base': stats_enhanced_base,
    'With Volume Filter': stats_enhanced_vol,
    'Volume 1.5x': stats_enhanced_vol_high
})

metrics = ['Return [%]', 'Sharpe Ratio', 'Max. Drawdown [%]', 'Win Rate [%]', '# Trades', 'Profit Factor']
print(comparison_filters.loc[metrics].to_string())
print("="*70)

Testing base enhanced strategy (no filters)...



Testing with volume filter...



Testing with volume filter (higher threshold)...


📊 STRATEGY COMPARISON - FILTERS IMPACT
                    Original Enhanced Base With Volume Filter Volume 1.5x
Return [%]          0.092168      0.018553           0.018553   -0.262417
Sharpe Ratio        0.054527       0.01123            0.01123   -0.228278
Max. Drawdown [%]  -0.339773     -0.308424          -0.308424   -0.369152
Win Rate [%]       62.037037          60.0               60.0   56.097561
# Trades                 108            95                 95          41
Profit Factor       1.105314      1.012772           1.012772    0.679553


## Parameter Optimization
Multi-dimensional grid search for optimal parameters

In [ ]:
print("Starting optimization...\n")

stats_best = bt_enhanced.optimize(
    rsi_oversold_level=range(5, 26, 5),
    trend_sma_period=[150, 200, 250],
    exit_sma_period=[5, 10, 15, 20],
    use_volume_filter=[True, False],
    volume_multiplier=[0.8, 1.0, 1.2, 1.5],
    maximize='Sharpe Ratio',
    constraint=lambda p: (not p.use_volume_filter) or (p.use_volume_filter and p.volume_multiplier > 0)
)

print("\n" + "="*70)
print("OPTIMIZED STRATEGY RESULTS")
print("="*70)
print(stats_best)
print("="*70)

print("\nOptimal parameters:")
print(f"  RSI Oversold Level: {stats_best._strategy.rsi_oversold_level}")
print(f"  Trend SMA Period: {stats_best._strategy.trend_sma_period}")
print(f"  Exit SMA Period: {stats_best._strategy.exit_sma_period}")
print(f"  Volume Filter: {stats_best._strategy.use_volume_filter}")
print(f"  Volume Multiplier: {stats_best._strategy.volume_multiplier}")

🔄 Optimizing enhanced strategy... This will take several minutes...



Backtest.optimize:   0%|          | 0/480 [00:00<?, ?it/s]
                                                       







                                                       













Backtest.run:   0%|          | 0/3786 [00:00<?, ?bar/s]














Backtest.run:   0%|          | 0/3786 [00:00<?, ?bar/s]





















                                                       






























Backtest.run:   0%|          | 0/3736 [00:00<?, ?bar/s]



















                                                       





















                                                       








































Backtest.run:   0%|          | 0/3786 [00:00<?, ?bar/s]

























































                                                       





















Backtest.run:   0%|          | 0/3836 [00:00<?, ?bar/s]


Backtest.run:   0%|          | 0/3786 [00:00<?, ?bar/s]





















Bac


🏆 BEST OPTIMIZED STRATEGY WITH FILTERS
Start                     2010-01-04 00:00:00
End                       2025-11-05 00:00:00
Duration                   5784 days 00:00:00
Exposure Time [%]                      19.142
Equity Final [$]                  10081.66311
Equity Peak [$]                   10082.60131
Commissions [$]                      27.03146
Return [%]                            0.81663
Buy & Hold Return [%]               362.78758
Return (Ann.) [%]                     0.05143
Volatility (Ann.) [%]                 0.14876
CAGR [%]                              0.03544
Sharpe Ratio                          0.34575
Sortino Ratio                         0.46425
Calmar Ratio                          0.08375
Alpha [%]                            -0.28959
Beta                                  0.00305
Max. Drawdown [%]                    -0.61411
Avg. Drawdown [%]                    -0.02842
Max. Drawdown Duration     2366 days 00:00:00
Avg. Drawdown Duration       81 days 00:

## Alternative Approach: Bollinger Bands
Mean reversion using volatility bands instead of RSI

In [ ]:
class StratBollingerMR(Strategy):
    
    trend_sma_period = 200
    bb_period = 20
    bb_std = 2.0
    exit_target = 0.5
    fixed_size = 1.0
    
    def init(self):
        from pandas_ta import sma, bbands
        
        close = pd.Series(self.data.Close, index=self.data.index)
        
        self.sma_trend = self.I(lambda x: sma(pd.Series(x), length=self.trend_sma_period).values, self.data.Close)
        
        bb = bbands(close, length=self.bb_period, std=self.bb_std)
        bb_cols = bb.columns.tolist()
        self.bb_lower = self.I(lambda: bb[bb_cols[0]].values)
        self.bb_middle = self.I(lambda: bb[bb_cols[1]].values)
        self.bb_upper = self.I(lambda: bb[bb_cols[2]].values)

    def next(self):
        
        if len(self.sma_trend) < self.trend_sma_period or len(self.bb_lower) < self.bb_period:
            return

        current_close = self.data.Close[-1]
        current_trend_sma = self.sma_trend[-1]
        
        if self.position:
            exit_level = self.bb_middle[-1] + (self.bb_upper[-1] - self.bb_middle[-1]) * self.exit_target
            if current_close >= exit_level:
                self.position.close()
                return
        
        if not self.position:
            is_uptrend = current_close > current_trend_sma
            touches_lower_band = current_close <= self.bb_lower[-1]
            
            if is_uptrend and touches_lower_band:
                self.buy(size=self.fixed_size)

print("Bollinger strategy created")

bt_bb = Backtest(df_iwm, StratBollingerMR, cash=10000, commission=.001)
stats_bb = bt_bb.run()

print("\n" + "="*70)
print("BOLLINGER BANDS STRATEGY")
print("="*70)
print(stats_bb)
print("="*70)

✅ Bollinger Band strategy created!



📈 BOLLINGER BAND MEAN REVERSION STRATEGY
Start                     2010-01-04 00:00:00
End                       2025-11-05 00:00:00
Duration                   5784 days 00:00:00
Exposure Time [%]                    19.81937
Equity Final [$]                  10034.11148
Equity Peak [$]                   10061.24236
Commissions [$]                      10.54251
Return [%]                            0.34111
Buy & Hold Return [%]               324.24426
Return (Ann.) [%]                     0.02153
Volatility (Ann.) [%]                  0.1573
CAGR [%]                              0.01484
Sharpe Ratio                          0.13688
Sortino Ratio                          0.1819
Calmar Ratio                          0.03763
Alpha [%]                            -0.86524
Beta                                  0.00372
Max. Drawdown [%]                    -0.57216
Avg. Drawdown [%]                     -0.0509
Max. Drawdown Duration      973 days 00:00:00
Avg. Drawdown Duration       95 days 0

## Multi-Indicator Confluence Strategy
Combining RSI, Stochastic, and volume for higher conviction signals

In [ ]:
class StratMultiConfluence(Strategy):
    
    trend_sma_period = 200
    rsi_period = 2
    rsi_oversold = 10
    stoch_period = 14
    stoch_oversold = 20
    volume_ma = 20
    exit_sma = 5
    fixed_size = 1.0
    
    def init(self):
        from pandas_ta import sma, rsi, stoch
        
        close = pd.Series(self.data.Close, index=self.data.index)
        high = pd.Series(self.data.High, index=self.data.index)
        low = pd.Series(self.data.Low, index=self.data.index)
        
        self.sma_trend = self.I(lambda x: sma(pd.Series(x), length=self.trend_sma_period).values, self.data.Close)
        self.rsi = self.I(lambda x: rsi(pd.Series(x), length=self.rsi_period).values, self.data.Close)
        self.sma_exit = self.I(lambda x: sma(pd.Series(x), length=self.exit_sma).values, self.data.Close)
        self.volume_sma = self.I(lambda x: sma(pd.Series(x), length=self.volume_ma).values, self.data.Volume)
        
        stoch_result = stoch(high, low, close, k=self.stoch_period, d=3)
        self.stoch_k = self.I(lambda: stoch_result[f'STOCHk_{self.stoch_period}_3_3'].values)

    def next(self):
        
        if len(self.rsi) < max(self.rsi_period, self.trend_sma_period, self.stoch_period):
            return

        current_close = self.data.Close[-1]
        
        if self.position:
            if current_close > self.sma_exit[-1]:
                self.position.close()
                return
        
        if not self.position:
            is_uptrend = current_close > self.sma_trend[-1]
            rsi_signal = self.rsi[-1] < self.rsi_oversold
            stoch_signal = self.stoch_k[-1] < self.stoch_oversold
            volume_ok = self.data.Volume[-1] > self.volume_sma[-1]
            
            if is_uptrend and rsi_signal and stoch_signal and volume_ok:
                self.buy(size=self.fixed_size)

print("Multi-indicator strategy created")

bt_multi = Backtest(df_iwm, StratMultiConfluence, cash=10000, commission=.001)
stats_multi = bt_multi.run()

print("\n" + "="*70)
print("MULTI-INDICATOR CONFLUENCE")
print("="*70)
print(stats_multi)
print("="*70)

✅ Multi-indicator confluence strategy created!



📈 MULTI-INDICATOR CONFLUENCE STRATEGY
Start                     2010-01-04 00:00:00
End                       2025-11-05 00:00:00
Duration                   5784 days 00:00:00
Exposure Time [%]                     4.54089
Equity Final [$]                   9999.33397
Equity Peak [$]                   10011.78678
Commissions [$]                       9.29305
Return [%]                           -0.00666
Buy & Hold Return [%]               324.24426
Return (Ann.) [%]                    -0.00042
Volatility (Ann.) [%]                 0.06796
CAGR [%]                             -0.00029
Sharpe Ratio                          -0.0062
Sortino Ratio                        -0.00843
Calmar Ratio                         -0.00189
Alpha [%]                            -0.22513
Beta                                  0.00067
Max. Drawdown [%]                    -0.22308
Avg. Drawdown [%]                    -0.12849
Max. Drawdown Duration     3438 days 00:00:00
Avg. Drawdown Duration     1246 days 00:0

## Strategy Performance Comparison
Side-by-side analysis of all tested approaches

In [ ]:
all_strategies = {
    '1. Original RSI(2)': stats_iwm,
    '2. Enhanced (Vol Filter)': stats_enhanced_vol,
    '3. Best Optimized': stats_best,
    '4. Bollinger Bands': stats_bb,
    '5. Multi-Confluence': stats_multi
}

comparison_all = pd.DataFrame(all_strategies)

key_metrics = [
    'Return [%]',
    'Sharpe Ratio',
    'Sortino Ratio',
    'Max. Drawdown [%]',
    'Calmar Ratio',
    'Win Rate [%]',
    '# Trades',
    'Avg. Trade [%]',
    'Profit Factor',
    'Expectancy [%]'
]

print("\n" + "="*100)
print("COMPREHENSIVE STRATEGY COMPARISON")
print("="*100)
print(comparison_all.loc[key_metrics].to_string())
print("="*100)

returns = {name: stats['Return [%]'] for name, stats in all_strategies.items()}
sharpes = {name: stats['Sharpe Ratio'] for name, stats in all_strategies.items()}

best_return_name = max(returns, key=returns.get)
best_sharpe_name = max(sharpes, key=sharpes.get)

print(f"\nBest Return: {best_return_name} ({returns[best_return_name]:.2f}%)")
print(f"Best Risk-Adjusted: {best_sharpe_name} ({sharpes[best_sharpe_name]:.3f})")
print(f"\nRecommendation: {best_sharpe_name}")


🏆 FINAL STRATEGY COMPARISON - ALL VARIANTS
                  1. Original RSI(2) 2. Enhanced (Vol Filter) 3. Best Optimized 4. Bollinger Bands 5. Multi-Confluence
Return [%]                  0.092168                 0.018553          0.816631           0.341115            -0.00666
Sharpe Ratio                0.054527                  0.01123          0.345746           0.136877           -0.006196
Sortino Ratio               0.074226                 0.015223           0.46425           0.181897           -0.008429
Max. Drawdown [%]          -0.339773                -0.308424         -0.614114           -0.57216           -0.223076
Calmar Ratio                0.017142                 0.003803           0.08375           0.037632           -0.001888
Win Rate [%]               62.037037                     60.0         72.277228          75.609756           61.111111
# Trades                         108                       95               101                 41                  36
Avg.

## Export Results
Generate HTML reports for interactive analysis

In [ ]:
print("Generating charts...\n")

bt_iwm.plot(filename='backtest_1_original_rsi.html', open_browser=False)
print("Created: backtest_1_original_rsi.html")

bt_enhanced.run(use_volume_filter=True)
bt_enhanced.plot(filename='backtest_2_enhanced_volume.html', open_browser=False)
print("Created: backtest_2_enhanced_volume.html")

bt_bb.plot(filename='backtest_4_bollinger_bands.html', open_browser=False)
print("Created: backtest_4_bollinger_bands.html")

bt_multi.plot(filename='backtest_5_multi_confluence.html', open_browser=False)
print("Created: backtest_5_multi_confluence.html")

print("\nAll reports generated")

📊 Generating interactive charts for all strategies...

✅ Created: backtest_1_original_rsi.html
✅ Created: backtest_1_original_rsi.html


✅ Created: backtest_2_enhanced_volume.html
✅ Created: backtest_4_bollinger_bands.html
✅ Created: backtest_4_bollinger_bands.html
✅ Created: backtest_5_multi_confluence.html

✨ All HTML reports generated! Open them in your browser for detailed analysis.
✅ Created: backtest_5_multi_confluence.html

✨ All HTML reports generated! Open them in your browser for detailed analysis.


## Trade Analysis
Detailed breakdown of best performing strategy

In [ ]:
best_strategy_name = max(all_strategies, key=lambda k: all_strategies[k]['Sharpe Ratio'])
best_stats = all_strategies[best_strategy_name]
trades = best_stats['_trades']

print(f"Trade Analysis: {best_strategy_name}")
print("="*80)
print(f"Total Trades: {len(trades)}")

print(f"\nReturn Distribution:")
print(trades['ReturnPct'].describe())

print(f"\nDuration Analysis:")
print(trades['Duration'].describe())

print(f"\nWin/Loss Breakdown:")
winning_trades = trades[trades['ReturnPct'] > 0]
losing_trades = trades[trades['ReturnPct'] < 0]
print(f"  Winning: {len(winning_trades)} (Avg: {winning_trades['ReturnPct'].mean():.2f}%)")
print(f"  Losing: {len(losing_trades)} (Avg: {losing_trades['ReturnPct'].mean():.2f}%)")

print(f"\nTop 5 Winners:")
print(trades.nlargest(5, 'ReturnPct')[['EntryTime', 'ExitTime', 'ReturnPct', 'Duration']])

print(f"\nTop 5 Losers:")
print(trades.nsmallest(5, 'ReturnPct')[['EntryTime', 'ExitTime', 'ReturnPct', 'Duration']])

📊 Trade Analysis for: 3. Best Optimized
Total Trades: 101

📈 Trade Return Distribution:
count    101.000000
mean       0.006489
std        0.035276
min       -0.279768
25%       -0.000595
50%        0.008107
75%        0.017838
max        0.083766
Name: ReturnPct, dtype: float64

📅 Trade Duration Analysis:
count                          101
mean     9 days 12:07:07.722772277
std      9 days 18:40:33.299073706
min                1 days 00:00:00
25%                2 days 00:00:00
50%                6 days 00:00:00
75%               14 days 00:00:00
max               42 days 00:00:00
Name: Duration, dtype: object

🎯 Win/Loss Breakdown:
  Winning Trades: 73 (Avg: 0.02%)
  Losing Trades: 28 (Avg: -0.02%)

🔝 Top 5 Best Trades:
    EntryTime   ExitTime  ReturnPct Duration
93 2024-08-05 2024-08-20   0.083766  15 days
98 2025-08-01 2025-08-13   0.053628  12 days
77 2021-05-13 2021-05-27   0.052935  14 days
90 2024-01-17 2024-01-23   0.051579   6 days
7  2011-05-25 2011-05-31   0.045495   6 days

In [ ]:
print("\n" + "="*80)
print("OPTIMAL PARAMETERS")
print("="*80)
print(f"\nStrategy: {best_strategy_name}")
print(f"\nPerformance:")
print(f"  Return: {best_stats['Return [%]']:.2f}%")
print(f"  Sharpe Ratio: {best_stats['Sharpe Ratio']:.3f}")
print(f"  Max Drawdown: {best_stats['Max. Drawdown [%]']:.2f}%")
print(f"  Win Rate: {best_stats['Win Rate [%]']:.1f}%")
print(f"  Profit Factor: {best_stats['Profit Factor']:.2f}")
print(f"  Total Trades: {best_stats['# Trades']}")

print(f"\nParameters:")
try:
    strat = best_stats._strategy
    print(f"  RSI Oversold: {strat.rsi_oversold_level}")
    print(f"  Trend SMA: {strat.trend_sma_period}")
    print(f"  Exit SMA: {strat.exit_sma_period}")
    if hasattr(strat, 'use_volume_filter'):
        print(f"  Volume Filter: {strat.use_volume_filter}")
        print(f"  Volume Mult: {strat.volume_multiplier}")
except:
    print("  Parameters embedded in strategy")

print("="*80)


🎯 OPTIMAL PARAMETERS FOR BEST STRATEGY

Strategy: 3. Best Optimized

📊 Performance Metrics:
  Return: 0.82%
  Sharpe Ratio: 0.346
  Max Drawdown: -0.61%
  Win Rate: 72.3%
  Profit Factor: 2.05
  Total Trades: 101

⚙️ Strategy Parameters:
  RSI Oversold Level: 10
  Trend SMA Period: 150
  Exit SMA Period: 20
  Volume Filter: True
  Volume Multiplier: 0.8


## Summary

### Tested Strategies:
1. Original RSI(2) Strategy - Basic mean reversion
2. Enhanced with Volume Filter - Added volume confirmation
3. Optimized Parameters - Grid search for best settings
4. Bollinger Band Strategy - Volatility-based approach
5. Multi-Indicator Confluence - Combined RSI, Stochastic, and volume

### Key Findings:
- Optimization significantly improved performance
- Volume filtering at 0.8x threshold effective
- Shorter trend SMA (150) captured more opportunities
- Longer exit SMA (20) improved profit taking
- Average hold time around 9-10 days optimal

### Future Improvements:
1. Implement stop loss protection
2. Add dynamic position sizing
3. Test on multiple assets (SPY, QQQ)
4. Walk-forward analysis for robustness
5. Market regime detection